# Hotdog / Not Hotdog Classification - transfer learning w/ ResNet50

In [1]:
import numpy as np

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

from keras.applications import resnet
from keras.applications.resnet import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import (
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
)
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
TRAIN_DATA_DIR = "data/train"
VALIDATION_DATA_DIR = "data/test"

BOTTLENECK_TRAIN_PATH = "auxiliary/resnet_bottleneck_train.npy"
BOTTLENECK_VALIDATION_PATH = "auxiliary/resnet_bottleneck_validation.npy"
FLATTENED_TOP_MODEL_PATH = "auxiliary/resnet_bottleneck_flattened_fc_model.h5"
POOLED_TOP_MODEL_PATH = "auxiliary/resnet_bottleneck_pooled_fc_model.h5"

TRAIN_CLASS_SIZE = 249
VALIDATION_CLASS_SIZE = 250

IMG_WIDTH, IMG_HEIGHT = 150, 150

In [3]:
def get_batches(
    path,
    datagen=ImageDataGenerator(preprocessing_function=resnet.preprocess_input),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=16,
    class_mode=None,
    shuffle=False,
    save_to_dir=None,
):
    generator = datagen.flow_from_directory(
        path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=shuffle,
        save_to_dir=save_to_dir,
    )
    return generator

In [4]:
resnet_model = ResNet50(include_top=False, weights="imagenet")

train_generator = get_batches(
    TRAIN_DATA_DIR, batch_size=TRAIN_CLASS_SIZE, save_to_dir="auxiliary/preview_train"
)
bottleneck_features_train = resnet_model.predict_generator(train_generator, steps=2)
np.save(open(BOTTLENECK_TRAIN_PATH, "wb"), bottleneck_features_train)

validation_generator = get_batches(
    VALIDATION_DATA_DIR,
    batch_size=VALIDATION_CLASS_SIZE,
    save_to_dir="auxiliary/preview_validation",
)
bottleneck_features_validation = resnet_model.predict_generator(validation_generator, steps=2)
np.save(open(BOTTLENECK_VALIDATION_PATH, "wb"), bottleneck_features_validation)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [5]:
train_data = np.load(open(BOTTLENECK_TRAIN_PATH, "rb"))
train_labels = np.array([0] * TRAIN_CLASS_SIZE + [1] * TRAIN_CLASS_SIZE)

validation_data = np.load(open(BOTTLENECK_VALIDATION_PATH, "rb"))
validation_labels = np.array([0] * VALIDATION_CLASS_SIZE + [1] * VALIDATION_CLASS_SIZE)

## ResNet 4D output flattened

In [6]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=RMSprop(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

checkpoint = ModelCheckpoint(
    FLATTENED_TOP_MODEL_PATH,
    monitor="val_acc",
    verbose=1,
    save_best_only=True,
    mode="max",
)

model.fit(
    train_data,
    train_labels,
    epochs=100,
    validation_data=(validation_data, validation_labels),
    callbacks=[checkpoint],
)

Train on 498 samples, validate on 500 samples
Epoch 1/100
498/498 [==============================] - 4s 9ms/step - loss: 3.4375 - acc: 0.6727 - val_loss: 1.3484 - val_acc: 0.7580

Epoch 00001: val_acc improved from -inf to 0.75800, saving model to auxiliary/resnet_bottleneck_flattened_fc_model.h5
Epoch 2/100
498/498 [==============================] - 3s 6ms/step - loss: 0.7427 - acc: 0.8675 - val_loss: 0.4857 - val_acc: 0.8940

Epoch 00002: val_acc improved from 0.75800 to 0.89400, saving model to auxiliary/resnet_bottleneck_flattened_fc_model.h5
Epoch 3/100
498/498 [==============================] - 3s 6ms/step - loss: 0.2398 - acc: 0.9458 - val_loss: 0.5162 - val_acc: 0.8700

Epoch 00003: val_acc did not improve from 0.89400
Epoch 4/100
498/498 [==============================] - 3s 5ms/step - loss: 0.1539 - acc: 0.9679 - val_loss: 0.9222 - val_acc: 0.8480

Epoch 00004: val_acc did not improve from 0.89400
Epoch 5/100
498/498 [==============================] - 3s 5ms/step - loss: 0.07

## ResNet 4D output pooled

In [9]:
model = Sequential()
# model.add(GlobalAveragePooling2D())
model.add(GlobalMaxPooling2D())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=RMSprop(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

checkpoint = ModelCheckpoint(
    POOLED_TOP_MODEL_PATH, monitor="val_acc", verbose=1, save_best_only=True, mode="max"
)

model.fit(
    train_data,
    train_labels,
    epochs=100,
    validation_data=(validation_data, validation_labels),
    callbacks=[checkpoint],
)

Train on 498 samples, validate on 500 samples
Epoch 1/100
498/498 [==============================] - 2s 4ms/step - loss: 2.9542 - acc: 0.6145 - val_loss: 1.0943 - val_acc: 0.7580

Epoch 00001: val_acc improved from -inf to 0.75800, saving model to auxiliary/resnet_bottleneck_pooled_fc_model.h5
Epoch 2/100
498/498 [==============================] - 0s 597us/step - loss: 1.6441 - acc: 0.7329 - val_loss: 1.0121 - val_acc: 0.7760

Epoch 00002: val_acc improved from 0.75800 to 0.77600, saving model to auxiliary/resnet_bottleneck_pooled_fc_model.h5
Epoch 3/100
498/498 [==============================] - 0s 579us/step - loss: 1.1577 - acc: 0.8072 - val_loss: 0.9137 - val_acc: 0.8000

Epoch 00003: val_acc improved from 0.77600 to 0.80000, saving model to auxiliary/resnet_bottleneck_pooled_fc_model.h5
Epoch 4/100
498/498 [==============================] - 0s 579us/step - loss: 0.9473 - acc: 0.8394 - val_loss: 0.6908 - val_acc: 0.8260

Epoch 00004: val_acc improved from 0.80000 to 0.82600, saving

## Results

- Best checkpointed accuracy with Flattening - ~91%
- With Global Average Pooling - ~89%
- With Global Max Pooling - ~91%

Models trained on top of ResNet50 significantly outperformed those we trained on top VGG16.